# LSTM

Implementing LSTM for IMDB data. For data analysis, look at Logistic_Regression.ipynb

In [1]:
import imdb_functions # For easy data load.
import keras
import numpy as np
import pandas as pd
import sklearn as sk

from keras.datasets import imdb # Easier way to implement
from keras.layers import LSTM, Dense, Input, Embedding, Reshape
from keras.models import Model, save_model, load_model
from keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

Using TensorFlow backend.


In [8]:
(X_train,y_train),(X_test,y_test)=imdb.load_data(num_words=5000)

In [9]:
len(X_train), len(y_train),len(X_test), len(y_test)
# imdb1[0]

(25000, 25000, 25000, 25000)

In [10]:
max_review_length = 1024
X_train = pad_sequences(X_train, maxlen=max_review_length, padding='post')
X_test = pad_sequences(X_test, maxlen=max_review_length)

In [11]:
max([i.max() for i in X_train])

4999

In [12]:
X_train.shape, y_train.shape

((25000, 1024), (25000,))

In [14]:
# Model setup
input_nodes= Input(shape=(X_train.shape[1],))
# x = Dense(64, activation='relu', kernel_initializer='random_uniform')(input_nodes)
# e = Embedding(output_dim=64, input_dim=X_train.shape[1],embeddings_initializer='glorot_uniform', input_length=10)(input_nodes)
e = Embedding(5000,
              16,
#               weights=[embedding_matrix],
              input_length=X_train.shape[1],
              trainable=False)(input_nodes)
# x = Reshape(target_shape=())(input_nodes)
lstm=LSTM(32)(e)
# x = Dense(16, activation='relu')
output_nodes=Dense(1, activation='sigmoid')(lstm)

#Build model
model = Model(inputs=input_nodes, outputs=output_nodes)
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, batch_size=32, epochs=20)

Epoch 1/20
 2464/25000 [=>............................] - ETA: 12:40 - loss: 0.6932 - acc: 0.5118